In [1]:
import numpy as np
import librosa
import pandas as pd
import os
import freesound

print("Librosa version:", librosa.__version__)
print("Numpy version:", np.__version__)
print("Pandas version:", pd.__version__)
#print(freesound.__version__)

Librosa version: 0.11.0
Numpy version: 2.1.3
Pandas version: 2.2.3


In [2]:
import soundata

# Inicializace datasetu
dataset = soundata.initialize("urbansound8k")

# Stáhne dataset, pokud ještě není
print("📦 Kontroluji dataset...")
dataset.download()  

# Načte metadata
dataset.validate()
clip_ids = dataset.clip_ids

print(f"🔍 V databázi je {len(clip_ids)} klipů.")

INFO: Downloading ['all', 'index']. Index is being stored in /home/cigi/.local/lib/python3.10/site-packages/soundata/datasets/indexes, and the rest of files in /home/cigi/sound_datasets/urbansound8k
INFO: [all] downloading UrbanSound8K.tar.gz


📦 Kontroluji dataset...


5.61GB [34:57, 2.87MB/s]                               
INFO: [index] downloading urbansound8k_index_1.0.json
1.15MB [00:01, 1.15MB/s]                            
100%|██████████| 8732/8732 [00:14<00:00, 610.71it/s]
INFO: Success: the dataset is complete and all files are valid.
INFO: --------------------


🔍 V databázi je 8732 klipů.


In [ ]:
# API Klíč
API_KEY = "VV7pMu2KzRHPQkBZAFeSg96sH4hPdHi6OMryNnCc"

In [ ]:
import freesound
import os

API_KEY = "VV7pMu2KzRHPQkBZAFeSg96sH4hPdHi6OMryNnCc"

client = freesound.FreesoundClient()
client.set_token(API_KEY, "token")

# Výstupní složka
os.makedirs("freesound_gunshots", exist_ok=True)

#  Vyhledání gunshot zvuků (s výpisem)
print("🔍 Hledám zvuky 'gunshot' s licencí CC0...")
results = list(client.text_search(
    query="gunshot",
    filter="license:cc-by OR license:cc0",
    fields="id,name,previews,license",
    page_size=20,
    sort="score"
))

#  Výpis výsledků
print(f"\n🧾 Nalezeno {len(results)} výsledků:\n")

for i, sound in enumerate(results):
    print(f"[{i+1}] {sound.name} | ID: {sound.id} | Licence: {sound.license}")

if len(results) == 0:
    print("⚠️ Žádné výsledky nenalezeny! Zkus jiný dotaz nebo zkontroluj API klíč.")
    exit()

#  Stažení zvuků
print("\n📥 Spouštím stahování zvuků...")
for i, sound in enumerate(results):
    safe_name = f"{sound.id}_{sound.name.replace(' ', '_')}.mp3"
    path = os.path.join("freesound_gunshots", safe_name)
    print(f"[{i+1}] ⬇️ Stahuji: {safe_name}")
    try:
        sound.retrieve_preview(path)
    except Exception as e:
        print(f"❌ Chyba při stahování {safe_name}: {e}")

print("\n✅ Hotovo! Zkontroluj složku 'freesound_gunshots'.")

In [ ]:
#Převod na WAV


from pydub import AudioSegment

INPUT_DIR = "freesound_gunshots"
OUTPUT_DIR = "freesound_gunshots_wav"
os.makedirs(OUTPUT_DIR, exist_ok=True)

for file in os.listdir(INPUT_DIR):
    if file.endswith(".mp3"):
        mp3_path = os.path.join(INPUT_DIR, file)
        wav_name = os.path.splitext(file)[0] + ".wav"
        wav_path = os.path.join(OUTPUT_DIR, wav_name)

        try:
            sound = AudioSegment.from_mp3(mp3_path)
            sound.export(wav_path, format="wav")
            print(f"✅ {file} → {wav_name}")
        except Exception as e:
            print(f"❌ Chyba u {file}: {e}")

print("🎉 Všechny soubory převedeny do WAV!")

In [ ]:
def extract_mfcc(path, n_mfcc=13, duration=4.0, sr=22050):
    y, _ = librosa.load(path, sr=sr, duration=duration)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return np.mean(mfcc.T, axis=0)  # průměr přes časové okno

# Kategorie z UrbanSound8K
target_labels = ['gun_shot']
negative_labels = ['children_playing', 'air_conditioner', 'dog_bark']

X = []
y = []

#  1. uprava z UrbanSound8K
for clip_id in dataset.clip_ids:
    clip = dataset.clip(clip_id)
    if clip.tags:
        label = clip.tags.labels[0]

        if label in target_labels + negative_labels:
            features = extract_mfcc(clip.audio_path)
            X.append(features)
            y.append(1 if label == 'gun_shot' else 0)

#  2. uprava z vlastních Freesound WAV souborů
FREESOUND_WAV_DIR = "freesound_gunshots_wav"

for filename in os.listdir(FREESOUND_WAV_DIR):
    if filename.endswith(".wav"):
        path = os.path.join(FREESOUND_WAV_DIR, filename)
        try:
            features = extract_mfcc(path)
            X.append(features)
            y.append(1)  #
        except Exception as e:
            print(f"❌ Chyba při extrakci {filename}: {e}")

#  Výsledný dataset
X = np.array(X)
y = np.array(y)

print("✅ Dataset hotový:", X.shape, y.shape)

✅ Dataset hotový: (5503, 13) (5503,)


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Rozdělení dat
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Trénink modelu
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Vyhodnocení
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.93      0.91       613
           1       0.91      0.84      0.87       488

    accuracy                           0.89      1101
   macro avg       0.89      0.89      0.89      1101
weighted avg       0.89      0.89      0.89      1101

